<a href="https://colab.research.google.com/github/zhappo/Analise-de-dados-CRMV/blob/main/CRMV_Analises_de_dados_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instruções de uso
---

Os dados serão carregados a partir de informações obtidas no SISCAD WEB, que deverão ser informadas ou carregadas quando solicitadas.

##1.Obtenção e preparação dos arquivso a serem carregados

Os arquivso devem ser obtidos do SISCAD WEB em formato de excel.
Após baixar os mesmos, abra retire a primeira linha em branco e salve eles em formato padrão do excel (xlsx)

#Início do programa

In [53]:
#@title Carrega as bibliotecas necessarias para o programa funcionar
#Bibliotecas para importar arquivo
from google.colab import files
import io

#para analisar dados
import pandas as pd

#para gerar graficos
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [54]:
#@title Selecione o arquivo de relatorio geral

relatorio_geral_file = files.upload()

Saving geral.xlsx to geral (1).xlsx


In [55]:
#@title Selecione o arquivo com os dados de estabelecimentos comerciais
#
comercio_file = files.upload()

Saving comercio.xlsx to comercio (1).xlsx


In [56]:
#@title Selecione o arquivo com os dados de estabelecimentos que possuem atividades praticas
#
pratica_file = files.upload()

Saving pratica.xlsx to pratica (1).xlsx


In [57]:
#@title Tratamento dos dados
'''
################################################################################
 ---------------------------CRIA OS DATAFRAMES BASE-----------------------------
################################################################################
Nessa seção serão criados os dataframes base para o levantamento dos 
quantitativos e subdivisões/filtragens

################################################################################
'''

#                 Pega o nome dos arquivos carregados

#Para o relatório geral
relatorio_geral_keys = list(relatorio_geral_file.keys())
relatorio_geral_filename = relatorio_geral_keys[0]

#Para comércio
comercio_keys = list(comercio_file.keys())
comercio_filename = comercio_keys[0]

#Para estabelecimentso com atividades pratica
pratica_keys = list(pratica_file.keys())
pratica_filename = pratica_keys[0] 


#           Cria dataframe a partir dos arquivos carregados

#Do relatorio geral 
relatorio_geral_DF = pd.read_excel(f'/content/{relatorio_geral_filename}')

#Do relatório comércio
comercio_DF = pd.read_excel(f'/content/{comercio_filename}')

#Do relatório prática
pratica_DF = pd.read_excel(f'/content/{pratica_filename}')


#                         Dataframes auxiliares  

#Sobre pessoa jurídica (pj)
pj_DF = relatorio_geral_DF.loc[relatorio_geral_DF['Classe'] == 'PJ'] #Geral
pj_atuantes_DF = pj_DF.loc[pj_DF['Atuante'] == 'Sim'] #Atuantes
pj_nao_atuantes_DF = pj_DF.loc[pj_DF['Atuante'] == 'Não'] #Não atuantes

#Sobre cadastros (PR ou CD)
cadastros_DF = relatorio_geral_DF.loc[relatorio_geral_DF['Classe'] != 'PJ']
cadastros_atuantes_DF = cadastros_DF.loc[cadastros_DF['Atuante'] == 'Sim']
cadastros_nao_atuantes_DF = cadastros_DF.loc[cadastros_DF['Atuante'] == 'Não']


#                 Levantamento de valores quantitativos

estabelecimentos_quantidade = relatorio_geral_DF.shape[0] #de estabelecimentos no sistema

pj_quantidade = pj_DF.shape[0] #de PJ
pj_atuantes_quantidade = pj_atuantes_DF.shape[0]#de PJ atuantes
pj_nao_atuantes_quantidade = pj_nao_atuantes_DF.shape[0] #de PJ não atuantes

cadastros_quantidade = cadastros_DF.shape[0] #quantidades de cadastros
cadastros_atuantes_quantidade = cadastros_atuantes_DF.shape[0] 
cadastros_nao_atuantes_quantidade = cadastros_nao_atuantes_DF.shape[0]

'''
################################################################################ 
------------------TRATAMENTO DOS DATAFRAMES COMERCIO E PRATICA------------------
################################################################################
Observamos que existem repetições quando se obtem, os relatorios de estabelecimento
que possuem atividades de comércio e/ou prática. Nesta seção iremos tratar
buscando remover as repetições, bem como criar novos dataframes auxiliares para 
auxiliar nolevantamento de quantitativos.
################################################################################
'''
#cria um datafrem dos resultados comuns
intercessao_DF = comercio_DF[comercio_DF.Registro.isin(pratica_DF.Registro)] 

#cria um dataframe do tipo "comercio_DF - pratica_DF", ou seja, retira do comercio_DF
#todas linhas que as informações na coluna 'Registro' são iguais as do pratica_DF

comercio_diferenca_DF = comercio_DF[~comercio_DF.Registro.isin(pratica_DF.Registro)] 

#Elimina as entradas repetidas com dataframe.drop_duplicates(), para obter o resultado real
comercio_sem_repeticao_DF = comercio_DF.drop_duplicates()
intercessao_sem_repeticao_DF = intercessao_DF.drop_duplicates()
comercio_diferenca_sem_repeticao_DF = comercio_diferenca_DF.drop_duplicates()
pratica_sem_repeticao_DF = pratica_DF.drop_duplicates()

#Levantamento de valores quantitativos

comercio_sem_repeticao_quantidade = comercio_sem_repeticao_DF.shape[0]
intercessao_sem_repeticao_quantidade = intercessao_sem_repeticao_DF.shape[0]
comercio_diferenca_sem_repeticao_quantidade = comercio_diferenca_sem_repeticao_DF.shape[0]
pratica_sem_repeticao_quantidade = pratica_sem_repeticao_DF.shape[0]

#Geração de gráficos

In [58]:
#@title Estabelcimentos por situação

#Graficos Sunburst

fig_por_situacao = make_subplots(rows=1, cols=2, specs=[[{'type':'sunburst'}, {'type':'sunburst'}]])

#Sunbusrt 1x1
fig_por_situacao.add_trace(go.Sunburst(
    labels=['Pessoa Jurídica', 'Atuantes','Não Atuantes' ],
    parents=['','Pessoa Jurídica','Pessoa Jurídica'],
    values=[pj_quantidade,pj_atuantes_quantidade ,pj_nao_atuantes_quantidade]),
              1, 1)
#Sunburst 1x2
fig_por_situacao.add_trace(go.Sunburst(
    labels=['Cadastros', 'Atuantes','Não Atuantes' ],
    parents=['','Cadastros','Cadastros'],
    values=[cadastros_quantidade,cadastros_atuantes_quantidade,cadastros_nao_atuantes_quantidade]),
              1, 2)

In [60]:
from numpy import size
#@title Visão Geral

fig =go.Figure(go.Sunburst(
    labels=[ "Estabelecimentos",
            "Pessoa Jurídica","Cadastros",
            "Atuantes", "Não atuantes",
            "Comércio",
            "Sem atividade veterinária","Com atividade veterinária"
                          
            ],
    parents=["",
             "Estabelecimentos", "Estabelecimentos",
             "Pessoa Jurídica","Pessoa Jurídica",
             "Atuantes",
             "Comércio","Comércio"
             ],
    values=[estabelecimentos_quantidade,
            pj_quantidade, cadastros_quantidade,
            pj_atuantes_quantidade,pj_nao_atuantes_quantidade,
            comercio_sem_repeticao_quantidade,
            comercio_diferenca_sem_repeticao_quantidade, intercessao_sem_repeticao_quantidade
            ],
    branchvalues="total",
))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0),
                  height=600,
                  )

fig.show()